# Running MCMC on made composit

In [ ]:
import numpy as np

from ase.build import bulk
from forge.core.database import DatabaseManager
from forge.workflows.mcmc import MonteCarloAlloySampler
from mace.calculators.mace import MACECalculator

# 1) Fetch or build your initial supercell
#atoms = bulk("V", "bcc", a=3.03) * (4, 4, 4)  # 128 atoms if 2 atoms/cell * 4^3
db_config = {
    'database': {
        'dbname': 'test_database',
        'user': 'myless',
        'password': 'vcrtiwzr',
        'host': 'database-vcrtiwzr.cfg4i4qmuc4m.us-east-1.rds.amazonaws.com',
        'port': 5432
    }
}
db_manager = DatabaseManager(config_dict=db_config)
# Optionally randomize the composition a bit, or retrieve from your DB
# [Your code here to randomize or fetch structure]

# 1.5) Get a structure from the database without a calculation attached to it 
structures = db_manager.find_structures_without_calculation(model_type="vasp-static")


/home/myless/.miniforge3/envs/forge/lib/python3.11/site-packages/e3nn/o3/_wigner.py:10: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.



In [ ]:
from forge.analysis.composition import CompositionAnalyzer
# Example composition
composition = {
    'V': 0.91515,
    'W': 0.021,
    'Cr': 0.02385,
    'Ti': 0.03,
    'Zr': 0.01
}

analyzer = CompositionAnalyzer()
atoms = analyzer.create_random_alloy(
    composition=composition,
    crystal_type='bcc',
    dimensions=[4, 4, 4],
    lattice_constant=3.01,
    balance_element='V',
    cubic=True
)

In [ ]:
diverse_atoms = []
# Using the Diverse Compositions from previous section, let's do MCMC on them
for comp in diverse_compositions:
    atoms = analyzer.create_random_alloy(
        composition=comp,
        crystal_type='bcc',
        dimensions=[4, 4, 4],
        lattice_constant=3.01,
        balance_element='V',
        cubic=True
    )
    diverse_atoms.append(atoms)

## Run the MCMC 

In [ ]:
import forge
from pathlib import Path
forge_root = Path(forge.__file__).parent

model_path = forge_root / "tests" / "resources" / "potentials" / "mace" / "gen_5_model_0-11-28_stagetwo.model"
# 2) Initialize your ML potential
calc = MACECalculator(model_paths=[model_path],
                      device="cuda",
                      default_dtype="float32",
                      enable_cueq=True)

fin_diverse_atoms = []
# 3) Setup Monte Carlo sampler
temperature = 1200.0
steps_per_atom = 100  # e.g., 30 swaps per atom
total_swaps = steps_per_atom * len(atoms) 

for atoms in diverse_atoms:
    mc_sampler = MonteCarloAlloySampler(
        atoms=atoms,
        calculator=calc,
        temperature=temperature,
        steps=total_swaps,
    )

    # 4) Run MC simulation
    print(f"Starting MC simulation for {atoms.get_chemical_formula()}")
    final_atoms = mc_sampler.run_mcmc()
    print("MC simulation complete.")
    fin_diverse_atoms.append(final_atoms)

/home/myless/.miniforge3/envs/forge/lib/python3.11/site-packages/mace/calculators/mace.py:135: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.

/home/myless/.miniforge3/envs/fo

Converting models to CuEq for acceleration
Starting MC simulation for Cr2Ti2V120W2Zr2
MC simulation complete.
Starting MC simulation for Cr6Ti11V102W6Zr3
MC simulation complete.
Starting MC simulation for Cr4Ti6V113W3Zr2
MC simulation complete.
Starting MC simulation for Cr3Ti5V115W3Zr2
MC simulation complete.
Starting MC simulation for Cr5Ti9V107W4Zr3
MC simulation complete.
Starting MC simulation for Cr3Ti3V117W2Zr3
MC simulation complete.
Starting MC simulation for Cr2Ti4V118W2Zr2
MC simulation complete.
Starting MC simulation for Cr4Ti7V111W4Zr2
MC simulation complete.
Starting MC simulation for Cr3Ti6V114W3Zr2
MC simulation complete.


In [ ]:
print(atoms)

Atoms(symbols='Cr4Ti4V115W3Zr2', pbc=True, cell=[12.04, 12.04, 12.04])


## Output them to an xyz file or to a folder with xyz fles 

In [ ]:
from ase.io import write
import os
output_dir = './scratch/data/final_diverse_atoms_t2'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
for atoms in fin_diverse_atoms:
    write(os.path.join(output_dir, f'{atoms.get_chemical_formula()}.xyz'), atoms)

In [ ]:
from ase.io import write

# write before and after atoms to xyz
write('./scratch/data/before_atoms.xyz', atoms)
write('./scratch/data/after_atoms.xyz', final_atoms)


In [ ]:
print(atoms)
print(final_atoms)

Atoms(symbols='Cr4Ti4V115W3Zr2', pbc=True, cell=[12.04, 12.04, 12.04])
Atoms(symbols='Cr4Ti4V115W3Zr2', pbc=True, cell=[12.04, 12.04, 12.04], calculator=MACECalculator(...))


In [ ]:
# save the final divser atoms to xyz
write('./scratch/data/final_diverse_atoms.xyz', fin_diverse_atoms)


## Optionally add them to the database

In [ ]:
# 5) Optionally add final_atoms to your DB
db_config = {
    'database': {
        'dbname': 'test_database',
        'user': 'myless',
        'password': '***',
        'host': 'database-vcrtiwzr.cfg4i4qmuc4m.us-east-1.rds.amazonaws.com',
        'port': 5432
    }
}
db_manager = DatabaseManager(config_dict=db_config)
structure_id = db_manager.add_structure(final_atoms, source_type="MC_Sampler")
print(f"Added MC-refined structure ID: {structure_id}")


# Visualizing MCMC Results

In [ ]:
from forge.analysis.composition import CompositionAnalyzer
from forge.workflows.mcmc import MonteCarloAlloySampler
from ase import Atoms
from forge.analysis.wc_sro import WarrenCowleyCalculator
from mace.calculators.mace import MACECalculator
import forge
import os
from pathlib import Path

forge_root = Path(forge.__file__).parent
# load the calculator
model_path = forge_root / "tests" / "resources" / "potentials" / "mace" / "gen_5_model_0-11-28_stagetwo.model"
calc = MACECalculator(model_paths=[model_path],
                      device="cuda",
                      default_dtype="float32",
                      enable_cueq=True)

# set the save directory
save_dir_path = './scratch/data/mcmc_results_big'
if not os.path.exists(save_dir_path):
    os.makedirs(save_dir_path)

# Example composition
composition = {
    'V': 0.91515,
    'W': 0.021,
    'Cr': 0.02385,
    'Ti': 0.03,
    'Zr': 0.01
}

analyzer = CompositionAnalyzer()
atoms = analyzer.create_random_alloy(
    composition=composition,
    crystal_type='bcc',
    dimensions=[24, 24, 24],
    lattice_constant=3.01,
    balance_element='V',
    cubic=True
)

# Create tracker settings
tracker_settings = {
    "energy_freq": 100,          # Record energy every 10 steps
    "wc_freq": 100,             # Record WC params every 50 steps
    "lattice_constant": 3.03,   # For default BCC shells
}

# Create and run MCMC
sampler = MonteCarloAlloySampler(
    atoms=atoms,
    calculator=calc,
    temperature=1000.0,
    steps=1000,
    tracker_settings=tracker_settings
)

final_atoms = sampler.run_mcmc()

# Plot and save results
sampler.tracker.plot_results(save_dir=save_dir_path, colormap='tab20')

In [ ]:
cu_calc = MACECalculator(model_paths=[model_path],
                      device="cuda",
                      default_dtype="float32",
                      enable_cueq=True)

test_atoms = atoms.copy()
test_atoms.calc = cu_calc
test_atoms.get_potential_energy()


/home/myless/.miniforge3/envs/forge/lib/python3.11/site-packages/mace/calculators/mace.py:135: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f=model_path, map_loc

Converting models to CuEq for acceleration


-6192.2119140625